In [ ]:
import robin_stocks as r
import datetime as dt
'''
This is a script that return a call option to sell given a stock
'''

#!!! Fill out username and password
username = '54chenyuan@gmail.com'
password = '54tianCAI!'
#!!!

login = r.login(username, password)

def returnRate(sym, price):
    opts = r.find_options_by_expiration(sym, "2020-09-04", "call")
    #print("we have {} options to check".format(len(opts)))
    dicts = []
    num = 0
    for rlt in opts:
        num = num + 1
        if(num>100): break
        try:
            strike_p = float(rlt['strike_price'])
            bid_price = float(rlt['bid_price'])
            #print("bid_price: {}, strike_price: {}\n".format(bid_price, strike_p))
            if bid_price>0.01 and (strike_p-price)>=2*bid_price:
                dic = {}
                dic["bid_price"] = bid_price
                dic["strike_price"] = strike_p
                #print("Appended")
                dicts.append(dic)
        except Exception as e: 
            print(e)
            pass
    dicts.sort(key=lambda x: x["bid_price"], reverse=True)
    if(len(dicts)>0):
        rate = dicts[0]['bid_price']/price
        return rate,dicts[0]['bid_price']*100,dicts[0]['strike_price']
    else:
        return None

#stks = r.find_instrument_data("")
stks = [
    {'symbol': 'PLAY', 'tradeable': True, 'type': 'stock'},
    {'symbol': 'RCL', 'tradeable': True, 'type': 'stock'},
    {'symbol': 'UAL', 'tradeable': True, 'type': 'stock'}
]

nums = len(stks)
counter = 0
print("{} stocks to be processed...".format(nums))
outs = []
for stk in stks:
    counter = counter + 1
    if counter%(nums/100) == 0:
        print("\n {}\% finished\n".format(counter/(nums/100)))
    try:
        if not stk['tradeable']: continue
        if not stk['type'] == 'stock': continue
        sym = stk['symbol']
        price = float(r.get_stock_quote_by_symbol(sym)['last_trade_price'])
        if(price>200 or price<5): continue
    
        print("\n------------Getting option info for: {}-------------\n".format(sym))
        rate, profit, strike_p = returnRate(sym, price)
        if rate is not None:
            dic = {}
            dic['symbol'] = sym
            dic['returnRate'] = rate
            dic['profit'] = profit
            dic['strike_p'] = strike_p
            dic['price'] = price
            outs.append(dic)
    except Exception as e: 
        print(e)

outs.sort(key=lambda x: x["returnRate"], reverse=True)
# write to a file:
print("\nWe have {} stocks, writing data to the file: stocks.txt, a moment please...\n".format(len(outs)))
with open("stocks.txt", "w") as f:
    for stk in outs:
        print("  writing {}...".format(stk['symbol']))
        line = "Return rate: {}, profits per month: {}, SYMBOL: {}, strike price: {}, stock price: {}".format(stk['returnRate'], stk['profit'], stk['symbol'], stk['strike_p'], stk['price'])
        f.write(line)
        f.write('\n')
print("Done, please check the file.")    